In [1]:
# %pip install highway-env

# Environment
import gymnasium as gym
# from stable_baselines3 import PPO
# from stable_baselines3.common.env_util import make_vec_env
# from stable_baselines3.common.vec_env import SubprocVecEnv
import torch
import torch.nn as nn
import highway_env
from highway_env.envs.parking_env import ParkingEnv
from typing import List, Tuple, Optional, Callable, TypeVar, Generic, Union, Dict, Text

import sys
from tqdm.notebook import trange
from utils import record_videos, show_videos


# Models and computation
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import namedtuple

# Visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import trange

# IO
from pathlib import Path

#HW 3
from gym.envs.registration import registry, register
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from torch import nn
from pathlib import Path
from easyrl.agents.ppo_agent import PPOAgent
from easyrl.configs import cfg
from easyrl.configs import set_config
from easyrl.configs.command_line import cfg_from_cmd
from easyrl.engine.ppo_engine import PPOEngine
from easyrl.models.categorical_policy import CategoricalPolicy
from easyrl.models.diag_gaussian_policy import DiagGaussianPolicy
from easyrl.models.mlp import MLP
from easyrl.models.value_net import ValueNet
from easyrl.runner.nstep_runner import EpisodicRunner
from easyrl.utils.common import set_random_seed
from easyrl.utils.gym_util import make_vec_env
from easyrl.utils.common import load_from_json
from base64 import b64encode
import numpy as np
import pprint
from gym import utils

from typing import Tuple, List, Dict
import os
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
import torch.nn.functional as F
import gym_minigrid
import numpy as np
import matplotlib.pyplot as plt
import gym
from tqdm.notebook import tqdm
from gym_minigrid.envs.doorkey import DoorKeyEnv
import pandas as pd
import random
import gym
import pprint
import pybullet as p
import pybullet_data as py_d
import airobot as ar
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pylab
from airobot import Robot
from airobot.utils.common import quat2euler
from airobot.utils.common import euler2quat
from gym import spaces
from gym.envs.registration import registry, register
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from torch import nn
from pathlib import Path
from easyrl.agents.ppo_agent import PPOAgent
from easyrl.configs import cfg
from easyrl.configs import set_config
from easyrl.configs.command_line import cfg_from_cmd
from easyrl.engine.ppo_engine import PPOEngine
from easyrl.models.categorical_policy import CategoricalPolicy
from easyrl.models.diag_gaussian_policy import DiagGaussianPolicy
from easyrl.models.mlp import MLP
from easyrl.models.value_net import ValueNet
from easyrl.runner.nstep_runner import EpisodicRunner
from easyrl.utils.common import set_random_seed
from easyrl.utils.gym_util import make_vec_env
from easyrl.utils.common import load_from_json
from base64 import b64encode
%matplotlib inline
%load_ext tensorboard
import warnings
warnings.filterwarnings("ignore")

/Users/noahwiley-class/.pyenv/versions/3.10.1/envs/6.8200/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
/Users/noahwiley-class/.pyenv/versions/3.10.1/envs/6.8200/lib/python3.10/site-packages/easyrl/envs/shmem_vec_env.py:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool: ctypes.c_bool}
pybullet build time: Mar 14 2024 23:37:35


In [ ]:
#Matching class def for URRobotGym(gym.Env)
class CustomParking(ParkingEnv):
    def __init__(self, config: dict = None, render_mode: Optional[str] = None):
        super().__init__(config, render_mode)
        super().define_spaces()
        
    @classmethod
    def default_config(cls) -> dict:
        config = super().default_config()
        config.update({
            "observation": {
                "type": "KinematicsGoal",
                "features": ['x', 'y', 'vx', 'vy', 'cos_h', 'sin_h'],
                "scales": [100, 100, 5, 5, 1, 1],
                "normalize": False
            },
            "action": {
                "type": "ContinuousAction"
            },
            "reward_weights": [1, 0.3, 0, 0, 0.02, 0.02],
            "success_goal_reward": 0.12,
            "collision_reward": -5,
            "steering_range": np.deg2rad(45),
            "simulation_frequency": 15,
            "policy_frequency": 5,
            "duration": 100,
            "screen_width": 600,
            "screen_height": 300,
            "centering_position": [0.5, 0.5],
            "scaling": 7,
            "controlled_vehicles": 1,
            "vehicles_count": 0,
            "add_walls": True
        })
        return config
    
    def compute_reward(self, achieved_goal, desired_goal, info: dict, p: float = 0.5) -> float:
        return super().compute_reward(achieved_goal, desired_goal, info, p)
    
    def _reward(self, action) -> float:
        return super()._reward(action)
        # this is what is in the super class:
        # obs = self.observation_type_parking.observe()
        # obs = obs if isinstance(obs, tuple) else (obs,)
        # reward = sum(self.compute_reward(agent_obs['achieved_goal'], agent_obs['desired_goal'], {}) for agent_obs in obs)
        # reward += self.config['collision_reward'] * sum(v.crashed for v in self.controlled_vehicles)
        # return reward
    
    def seed(self, seed=None):
        self.np_random, seed = utils.seeding.np_random(seed)
        return [seed]
module_name = __name__
# print(registry.all())
env_name = 'CustomParking-v0'
if env_name in registry.all():
    del registry[env_name]
register(
    id=env_name,
    entry_point=f'{module_name}:CustomParking',
)


In [4]:
def train_parking_ppo():
    set_config("ppo")
    cfg.alg.num_envs = 1
    cfg.alg.num_steps = 2048
    cfg.alg.max_steps = 1e6
    cfg.alg.deque_size = 20
    cfg.alg.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    cfg.alg.env_name = "parking-v0"
    cfg.alg.save_dir = Path.cwd().absolute().joinpath('data').as_posix()
    cfg.alg.save_dir += '/'
    setattr(cfg.alg, 'diff_cfg', dict(save_dir=cfg.alg.save_dir))
    
    print(f'====================================')
    print(f'      Device:{cfg.alg.device}')
    print(f'      Total number of steps:{cfg.alg.max_steps}')
    print(f'====================================')

    set_random_seed(cfg.alg.seed)

    env_kwargs = dict()

    env = make_vec_env(cfg.alg.env_name, 1, env_kwargs=env_kwargs)

    env.reset()

    ob_size = env.observation_space.shape[0]

    actor_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)

    critic_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)
    if isinstance(env.action_space, gym.spaces.Discrete):
        act_size = env.action_space.n
        actor = CategoricalPolicy(actor_body,
                                 in_features=64,
                                 action_dim=act_size)
    elif isinstance(env.action_space, gym.spaces.Box):
        act_size = env.action_space.shape[0]
        actor = DiagGaussianPolicy(actor_body,
                                   in_features=64,
                                   action_dim=act_size,
                                   tanh_on_dist=cfg.alg.tanh_on_dist,
                                   std_cond_in=cfg.alg.std_cond_in)
    else:
        raise TypeError(f'Unknown action space type: {env.action_space}')

    critic = ValueNet(critic_body, in_features=64)
    agent = PPOAgent(actor=actor, critic=critic, env=env)
    runner = EpisodicRunner(agent=agent, env=env)
    engine = PPOEngine(agent=agent,
                       runner=runner)
    engine.train()
    stat_info, raw_traj_info = engine.eval(render=False,
                                           save_eval_traj=True,
                                           eval_num=1,
                                           sleep_time=0.0)
    pprint.pprint(stat_info)
    return cfg.alg.save_dir, agent

In [5]:
train_parking_ppo()

[INFO][2024-04-30 11:52:28]: Alogrithm type:<class 'easyrl.configs.ppo_config.PPOConfig'>
[INFO][2024-04-30 11:52:28]: Creating 1 environments.


      Device:cpu
      Total number of steps:1000000.0


NameNotFound: Environment `parking` doesn't exist. Did you mean: `CarRacing`?

In [2]:
# DO NOT MODIFY THIS
def train_ppo(use_sparse_reward: bool = False, use_subgoal: bool = False, with_obstacle: bool = False,
              apply_collision_penalty: bool = False, push_exp: bool = False, max_steps: int = 200000) -> str:
    """
      Parameters:
      - use_sparse_reward (bool): Specifies whether to use sparse rewards in the environment.
      - use_subgoal (bool): Specifies whether to include subgoals in the environment.
      - with_obstacle (bool): Specifies whether to include obstacles in the environment.
      - apply_collision_penalty (bool): Specifies whether to apply a penalty for collisions.
      - push_exp (bool): Specifies whether to train on a pushing experiment. Changes the environment to 'URPusher-v1'.
      - max_steps (int): The maximum number of steps to train the agent.

      Returns:
      - str: The directory where training data and models are saved.
    """
    set_config('ppo')
    cfg.alg.num_envs = 1
    cfg.alg.episode_steps = 100
    cfg.alg.max_steps = max_steps
    cfg.alg.deque_size = 20
    cfg.alg.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    cfg.alg.env_name = "parking-v0"
    #cfg.alg.env_name = 'URPusher-v1' if push_exp else 'URReacher-v1'
    cfg.alg.save_dir = Path.cwd().absolute().joinpath('data').as_posix()
    cfg.alg.save_dir += '/'
    # if push_exp:
    #     cfg.alg.save_dir += 'push'
    # else:
    #     cfg.alg.save_dir += 'sparse' if use_sparse_reward else 'dense'
    #     cfg.alg.save_dir += f'_ob_{str(with_obstacle)}'
    #     cfg.alg.save_dir += f'_sg_{str(use_subgoal)}'
    #     cfg.alg.save_dir += f'_col_{str(apply_collision_penalty)}'
    setattr(cfg.alg, 'diff_cfg', dict(save_dir=cfg.alg.save_dir))

    print(f'====================================')
    print(f'      Device:{cfg.alg.device}')
    print(f'      Total number of steps:{cfg.alg.max_steps}')
    print(f'====================================')

    set_random_seed(cfg.alg.seed)
    env_kwargs = dict()
    # env_kwargs=dict(use_sparse_reward=use_sparse_reward,
    #                 with_obstacle=with_obstacle,
    #                 use_subgoal=use_subgoal,
    #                 apply_collision_penalty=apply_collision_penalty) if not push_exp else dict()
    print(cfg.alg.env_name)
    env = make_vec_env(cfg.alg.env_name,
                       cfg.alg.num_envs,
                       env_kwargs=env_kwargs)
    env.reset()
    ob_size = env.observation_space.shape[0]

    actor_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)

    critic_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)
    if isinstance(env.action_space, gym.spaces.Discrete):
        act_size = env.action_space.n
        actor = CategoricalPolicy(actor_body,
                                 in_features=64,
                                 action_dim=act_size)
    elif isinstance(env.action_space, gym.spaces.Box):
        act_size = env.action_space.shape[0]
        actor = DiagGaussianPolicy(actor_body,
                                   in_features=64,
                                   action_dim=act_size,
                                   tanh_on_dist=cfg.alg.tanh_on_dist,
                                   std_cond_in=cfg.alg.std_cond_in)
    else:
        raise TypeError(f'Unknown action space type: {env.action_space}')

    critic = ValueNet(critic_body, in_features=64)
    agent = PPOAgent(actor=actor, critic=critic, env=env)
    runner = EpisodicRunner(agent=agent, env=env)
    engine = PPOEngine(agent=agent,
                       runner=runner)
    engine.train()
    stat_info, raw_traj_info = engine.eval(render=False,
                                           save_eval_traj=True,
                                           eval_num=1,
                                           sleep_time=0.0)
    pprint.pprint(stat_info)
    return cfg.alg.save_dir, agent


In [3]:
test_dir, save_agent = train_ppo(use_sparse_reward=True, use_subgoal=False, with_obstacle=False, apply_collision_penalty=False)

[INFO][2024-04-30 11:49:16]: Alogrithm type:<class 'easyrl.configs.ppo_config.PPOConfig'>
[INFO][2024-04-30 11:49:16]: Creating 1 environments.


      Device:cpu
      Total number of steps:200000
parking-v0


NameNotFound: Environment `parking` doesn't exist. Did you mean: `CarRacing`?

In [ ]:
class Config:
    def __init__(self,
                score_threshold=0.93,
                discount=0.995,
                lr=1e-3,
                max_grad_norm=0.5,
                log_interval=10,
                max_episodes=500,
                gae_lambda=0.95,
                use_critic=False,
                clip_ratio=0.2,
                target_kl=0.01,
                train_ac_iters=5,
                use_discounted_reward=False,
                entropy_coef=0.01,
                use_gae=False):

        self.score_threshold = score_threshold # criterion for early stopping. If the rolling average reward (over the last 100 episodes) is greater than it, it ends.
        self.discount = discount # discount factor
        self.lr = lr # learning rate
        self.max_grad_norm = max_grad_norm # the maximum gradient norm (https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)
        self.log_interval = log_interval # logging interval
        self.max_episodes = max_episodes # the maximum number of episodes.
        self.use_critic = use_critic # whether to use critic or not.
        self.clip_ratio = clip_ratio # clip_ratio of PPO.
        self.target_kl = target_kl # target KL divergence for early stoping train_ac_iters for PPO
        self.train_ac_iters = train_ac_iters # how many time to train ac_model using current computed old_logps
        self.gae_lambda=gae_lambda # lambda in Generalized Advantage Estimation (GAE)
        self.use_discounted_reward=use_discounted_reward # whether use discounted reward or not.
        self.entropy_coef = entropy_coef # entropy coefficient for PPO
        self.use_gae = use_gae # whether to use GAE or not.

In [ ]:

n_cpu = 6
batch_size = 64
env = make_vec_env("parking-v0", n_envs=n_cpu, vec_env_cls=SubprocVecEnv)
model = PPO(
    "MultiInputPolicy",
    env,
    policy_kwargs=dict(net_arch=[dict(pi=[512, 128], vf=[512, 128])]),
    n_steps=batch_size * 12 // n_cpu,
    batch_size=batch_size,
    n_epochs=50,
    learning_rate=5e-4,
    gamma=0.95,
    verbose=0,
    tensorboard_log="parking_ppo/",
)
# Train the agent
model.learn(total_timesteps=int(3e4))
# Save the agent
model.save("parking_ppo/model")

model = PPO.load("parking_ppo/model")
# env = gym.make("parking-v0", render_mode = "rgb_array")
# for _ in range(5):
#     obs, info = env.reset()
#     done = truncated = False
#     while not (done or truncated):
#         action, _ = model.predict(obs)
#         obs, reward, done, truncated, info = env.step(action)
#         env.render()

In [ ]:
env = gym.make("parking-v0", render_mode="rgb_array")
obs, info = env.reset()
rendered = env.render()
print(rendered.shape)


In [ ]:
model = PPO.load("parking_ppo/model")
env = gym.make("parking-v0", render_mode="rgb_array")
env = record_videos(env)
obs, info = env.reset()
done = False
while not done:
    action = model.predict(obs)[0]
    obs, reward, done, truncated, info = env.step(action)
env.close()
show_videos()

In [ ]:
print(env)
# fig, ax = plt.subplots(ncols=1, figsize=(12, 5))
# ax.imshow(render) # , cmap=plt.get_cmap('gray')
# plt.show()